In [49]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("spotify-datalake") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1024M") \
    .config("spark.ui.port", "4061") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
sc = spark.sparkContext

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: SparkSession$ does not exist in the JVM

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [28]:
playlists_v1_path = '/shared/sampled/playlists_v1.json'
playlists_v1_df = spark.read.json(playlists_v1_path)
tracks_v1_path = '/shared/sampled/tracks_v1.json'
tracks_v1_df = spark.read.json(tracks_v1_path)

Task 1A:
- Silver layer:

In [29]:
from pyspark.sql import functions as F

song_info_df = tracks_v1_df.select(
    F.col("track_name").alias("name"),
    F.col("track_uri").alias("track_uri"),
    F.col("duration_ms").alias("duration_ms"),
    F.col("album_uri").alias("album_uri"),
    F.col("artist_uri").alias("artist_uri")
)

song_info_df.write.format("parquet").mode("overwrite").save("datalake/silver/song_info")

25/02/04 01:13:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [30]:
album_info_df = tracks_v1_df.select(
    F.col("album_name").alias("name"),
    F.col("album_uri").alias("album_uri"),
    F.col("artist_uri").alias("artist_uri")
).distinct()

album_info_df.write.format("parquet").mode("overwrite").save("datalake/silver/album_info")

In [31]:
artist_info_df = tracks_v1_df.select(
    F.col("artist_name").alias("name"),
    F.col("artist_uri").alias("artist_uri")
).distinct()

artist_info_df.write.format("parquet").mode("overwrite").save("datalake/silver/artist_info")

In [32]:
playlist_info_df = playlists_v1_df.select(
    F.col("name").alias("name"),
    F.col("pid").alias("pid"),
    F.col("description").alias("description"),
    F.col("collaborative").alias("is_collaborative")
)

playlist_info_df.write.format("parquet").mode("overwrite").save("datalake/silver/playlist_info")

In [33]:
playlist_tracks_df = tracks_v1_df.select(
    F.col("pid").alias("playlist_id"),
    F.col("pos").alias("position"),
    F.col("track_uri").alias("track_uri"),
    F.col("artist_uri").alias("artist_uri"),
    F.col("album_uri").alias("album_uri")
)

playlist_tracks_df.write.format("parquet").mode("overwrite").save("datalake/silver/playlist_tracks")

25/02/04 01:13:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

- Gold layer:

In [34]:
playlist_info_gold_df = tracks_v1_df.groupBy("pid").agg(
    F.count("track_uri").alias("num_tracks"),
    F.sum("duration_ms").alias("total_duration_ms"),
    F.countDistinct("artist_uri").alias("num_artists"),
    F.countDistinct("album_uri").alias("num_albums")
).join(playlist_info_df, "pid", "inner")

playlist_info_gold_df.write.format("parquet").mode("overwrite").save("datalake/gold/playlist_info")

In [35]:
song_info_df = song_info_df.withColumnRenamed("name", "song_name")
album_info_df = album_info_df.withColumnRenamed("name", "album_name")
artist_info_df = artist_info_df.withColumnRenamed("name", "artist_name")

playlist_tracks_gold_df = playlist_tracks_df.join(
    song_info_df, "track_uri", "inner"
).join(
    album_info_df, "album_uri", "inner"
).join(
    artist_info_df, "artist_uri", "inner"
).select(
    "playlist_id",
    "position",
    "song_name",
    "album_name",
    "artist_name"
)

Task 1B:

In [36]:
playlist_info_gold_df.write.format("json").mode("overwrite").save("datalake/gold_json/playlist_info")
playlist_tracks_gold_df.write.format("json").mode("overwrite").save("datalake/gold_json/playlist_tracks")

In [37]:
playlist_info_gold_df.write.format("parquet").mode("overwrite").save("datalake/gold_parquet/playlist_info")
playlist_tracks_gold_df.write.format("parquet").mode("overwrite").save("datalake/gold_parquet/playlist_tracks")

25/02/04 01:14:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/04 01:14:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [38]:
import time

# tempo de load em json
start_time = time.time()
json_df = spark.read.json("datalake/gold_json/playlist_info")
json_df.count()
json_load_time = time.time() - start_time

print(f"Tempo de carregamento (JSON): {json_load_time} segundos")

# tempo de load em parquet
start_time = time.time()
parquet_df = spark.read.parquet("datalake/gold_parquet/playlist_info")
parquet_df.count()
parquet_load_time = time.time() - start_time

print(f"Tempo de carregamento (Parquet): {parquet_load_time} segundos")

Tempo de carregamento (JSON): 0.2530093193054199 segundos
Tempo de carregamento (Parquet): 0.16831183433532715 segundos


**Task 2**

In [46]:
bronze_path = "datalake/bronze/"
silver_path = "datalake/silver/"
gold_path = "datalake/gold_parquet/"

def ingest_new_data(version):
    playlists = spark.read.json(f"/shared/sampled/playlists_{version}.json")
    tracks = spark.read.json(f"/shared/sampled/tracks_{version}.json")

    playlists.write.mode("append").parquet(bronze_path + "playlists/")
    tracks.write.mode("append").parquet(bronze_path + "tracks/")

    return playlists, tracks

def update_silver_layer(playlists_v, tracks_v):
    
    song_info = spark.read.parquet(silver_path + "song_info")
    album_info = spark.read.parquet(silver_path + "album_info")
    artist_info = spark.read.parquet(silver_path + "artist_info")
    playlist_info = spark.read.parquet(silver_path + "playlist_info")
    playlist_tracks = spark.read.parquet(silver_path + "playlist_tracks")

    new_song_info = tracks_v.select(
        F.col("track_name").alias("name"),
        F.col("track_uri").alias("track_uri"),
        F.col("duration_ms").alias("duration_ms"),
        F.col("album_uri").alias("album_uri"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()
    song_info = song_info.unionByName(new_song_info, allowMissingColumns=True).dropDuplicates(["track_uri"])

    new_album_info = tracks_v.select(
        F.col("album_name").alias("name"),
        F.col("album_uri").alias("album_uri"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()
    album_info = album_info.unionByName(new_album_info, allowMissingColumns=True).dropDuplicates(["album_uri"])

    new_artist_info = tracks_v.select(
        F.col("artist_name").alias("name"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()
    artist_info = artist_info.unionByName(new_artist_info, allowMissingColumns=True).dropDuplicates(["artist_uri"])

    new_playlist_info = playlists_v.select(
        F.col("name").alias("name"),
        F.col("pid").alias("pid"),
        F.col("description").alias("description"),
        F.col("collaborative").alias("is_collaborative")
    ).distinct()
    playlist_info = playlist_info.unionByName(new_playlist_info, allowMissingColumns=True).dropDuplicates(["pid"])

    new_playlist_tracks = tracks_v.select(
        F.col("pid").alias("playlist_id"),
        F.col("pos").alias("position"),
        F.col("track_uri").alias("track_uri"),
        F.col("artist_uri").alias("artist_uri"),
        F.col("album_uri").alias("album_uri")
    )
    playlist_tracks = playlist_tracks.unionByName(new_playlist_tracks, allowMissingColumns=True).dropDuplicates(["playlist_id", "track_uri"])

    playlist_info = playlist_info.withColumn(
        "name", F.when(F.col("pid") == "11992", "GYM WORKOUT").otherwise(F.col("name"))
    ).withColumn(
        "is_collaborative", F.when(F.col("pid") == "11992", F.lit(True).cast("string")).otherwise(F.col("is_collaborative"))
    )

    song_info.write.mode("overwrite").parquet(silver_path + "song_info")
    album_info.write.mode("overwrite").parquet(silver_path + "album_info")
    artist_info.write.mode("overwrite").parquet(silver_path + "artist_info")
    playlist_info.write.mode("overwrite").parquet(silver_path + "playlist_info")
    playlist_tracks.write.mode("overwrite").parquet(silver_path + "playlist_tracks")

def update_gold_layer():
    playlist_info = spark.read.parquet(silver_path + "playlist_info")
    song_info = spark.read.parquet(silver_path + "song_info")
    playlist_tracks = spark.read.parquet(silver_path + "playlist_tracks")

    playlists_gold = playlist_info.alias("pi") \
        .join(playlist_tracks.alias("pt"), F.col("pi.pid") == F.col("pt.playlist_id"), "left") \
        .join(song_info.alias("si"), F.col("pt.track_uri") == F.col("si.track_uri"), "left") \
        .groupBy("pi.pid", "pi.name", "pi.description") \
        .agg(
            F.count("pt.track_uri").alias("num_tracks"),
            F.sum("si.duration_ms").alias("total_duration_ms"),
            F.countDistinct("pt.artist_uri").alias("num_artists"),
            F.countDistinct("pt.album_uri").alias("num_albums")
        )

    playlist_tracks_gold = playlist_tracks.join(
        song_info_df, "track_uri", "inner"
    ).join(
        album_info_df, "album_uri", "inner"
    ).join(
        artist_info_df, "artist_uri", "inner"
    ).select(
        "playlist_id",
        "position",
        "song_name",
        "album_name",
        "artist_name"
    )

    playlists_gold.write.mode("overwrite").parquet(gold_path + "playlist_info")
    playlist_tracks_gold.write.mode("overwrite").parquet(gold_path + "playlist_tracks")


In [44]:
def run_pipeline(version_path):
    print(f"Iniciando ingestão dos dados de {version_path}...")
    playlists, tracks = ingest_new_data(version_path)

    print("Atualizando a camada Silver...")
    update_silver_layer(playlists, tracks)

    print("Atualizando a camada Gold...")
    update_gold_layer()

    print(f"Pipeline concluído para {version_path}!")

In [47]:
run_pipeline("v2")

Iniciando ingestão dos dados de v2...


25/02/04 01:38:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Silver...


25/02/04 01:38:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/04 01:38:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/04 01:38:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Gold...


25/02/04 01:38:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:38:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:38:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:38:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:38:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:38:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:38:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:38:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on

Pipeline concluído para v2!


In [48]:
run_pipeline("v3")

Iniciando ingestão dos dados de v3...


25/02/04 01:39:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Silver...


25/02/04 01:39:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/04 01:39:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/04 01:39:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Atualizando a camada Gold...


25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/02/04 01:40:01 WARN RowBasedKeyValueBatch: Calling spill() on

Pipeline concluído para v3!


**Task 3**

In [ ]:
bronze_path = "datalake/bronze/"
silver_path = "datalake/silver_delta/"
gold_path = "datalake/gold_delta/"

def ingest_new_data(version):
    playlists = spark.read.json(f"/shared/sampled/playlists_{version}.json")
    tracks = spark.read.json(f"/shared/sampled/tracks_{version}.json")

    playlists.write.format("delta").mode("append").save(bronze_path + "playlists/")
    tracks.write.format("delta").mode("append").save(bronze_path + "tracks/")

    return playlists, tracks

def update_silver_layer(playlists_v, tracks_v):

    song_info = spark.read.parquet(silver_path + "song_info")
    album_info = spark.read.parquet(silver_path + "album_info")
    artist_info = spark.read.parquet(silver_path + "artist_info")
    playlist_info = spark.read.parquet(silver_path + "playlist_info")
    playlist_tracks = spark.read.parquet(silver_path + "playlist_tracks")

    new_song_info = tracks_v.select(
        F.col("track_name").alias("name"),
        F.col("track_uri").alias("track_uri"),
        F.col("duration_ms").alias("duration_ms"),
        F.col("album_uri").alias("album_uri"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()
    song_info = song_info.unionByName(new_song_info, allowMissingColumns=True).dropDuplicates(["track_uri"])

    new_album_info = tracks_v.select(
        F.col("album_name").alias("name"),
        F.col("album_uri").alias("album_uri"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()
    album_info = album_info.unionByName(new_album_info, allowMissingColumns=True).dropDuplicates(["album_uri"])

    new_artist_info = tracks_v.select(
        F.col("artist_name").alias("name"),
        F.col("artist_uri").alias("artist_uri")
    ).distinct()
    artist_info = artist_info.unionByName(new_artist_info, allowMissingColumns=True).dropDuplicates(["artist_uri"])

    new_playlist_info = playlists_v.select(
        F.col("name").alias("name"),
        F.col("pid").alias("pid"),
        F.col("description").alias("description"),
        F.col("collaborative").alias("is_collaborative")
    ).distinct()
    playlist_info = playlist_info.unionByName(new_playlist_info, allowMissingColumns=True).dropDuplicates(["pid"])

    new_playlist_tracks = tracks_v.select(
        F.col("pid").alias("playlist_id"),
        F.col("pos").alias("position"),
        F.col("track_uri").alias("track_uri"),
        F.col("artist_uri").alias("artist_uri"),
        F.col("album_uri").alias("album_uri")
    )
    playlist_tracks = playlist_tracks.unionByName(new_playlist_tracks, allowMissingColumns=True).dropDuplicates(["playlist_id", "track_uri"])

    def merge_delta(table, new_data, join_key):
        table.alias("old").merge(
            new_data.alias("new"), f"old.{join_key} = new.{join_key}"
        ).whenNotMatchedInsertAll().execute()

    merge_delta(song_info, new_song_info, "track_uri")
    merge_delta(album_info, new_album_info, "album_uri")
    merge_delta(artist_info, new_artist_info, "artist_uri")
    merge_delta(playlist_info, new_playlist_info, "pid")
    merge_delta(playlist_tracks, new_playlist_tracks, "track_uri")
    
    playlist_info.update(
        condition=F.col("pid") == 11992,
        set={"name": F.lit("GYM WORKOUT"), "collaborative": F.lit(True)}
    )

def update_gold_layer():
    playlist_info = spark.read.format("delta").load(silver_path + "playlist_info")
    song_info = spark.read.format("delta").load(silver_path + "song_info")
    playlist_tracks = spark.read.format("delta").load(silver_path + "playlist_tracks")

    playlists_gold = playlist_info.alias("pi") \
        .join(playlist_tracks.alias("pt"), F.col("pi.pid") == F.col("pt.playlist_id"), "left") \
        .join(song_info.alias("si"), F.col("pt.track_uri") == F.col("si.track_uri"), "left") \
        .groupBy("pi.pid", "pi.name", "pi.description") \
        .agg(
            F.count("pt.track_uri").alias("num_tracks"),
            F.sum("si.duration_ms").alias("total_duration_ms"),
            F.countDistinct("pt.artist_uri").alias("num_artists"),
            F.countDistinct("pt.album_uri").alias("num_albums")
        )

    playlist_tracks_gold = playlist_tracks.join(
        song_info_df, "track_uri", "inner"
    ).join(
        album_info_df, "album_uri", "inner"
    ).join(
        artist_info_df, "artist_uri", "inner"
    ).select(
        "playlist_id",
        "position",
        "song_name",
        "album_name",
        "artist_name"
    )

    playlists_gold.write.format("delta").mode("overwrite").save(gold_path + "playlist_info")
    playlist_tracks_gold.write.format("delta").mode("overwrite").save(gold_path + "playlist_tracks")


In [ ]:
def run_pipeline(version_path):
    print(f"Iniciando ingestão dos dados de {version_path}...")
    playlists, tracks = ingest_new_data(version_path)

    print("Atualizando a camada Silver...")
    update_silver_layer(playlists, tracks)

    print("Atualizando a camada Gold...")
    update_gold_layer()

    print(f"Pipeline concluído para {version_path}!")

In [ ]:
run_pipeline("v2")

In [ ]:
run_pipeline("v3")